In [27]:
import pandas as pd
import numpy as np
from scipy import io
import plotly.express as px
import math
import re
import random as rnd
import json

In [28]:
sectors = [
    'Agriculture', 'Mining', 'Oil/Gas', 'Mining Support', 'Util', 'Const', 'Wood', 'Minerals', 'Primary Metals', 'Fabricated Metals',
    'Machinery', 'Computers', 'Electrical', 'Vehicles', 'Transport', 'Furniture',
    'Misc Mfg', 'Food Mfg', 'Textile', 'Apparel', 'Paper', 'Printing', 'Petroleum', 
    'Chemical', 'Plastics', 'Wholesale Trade', 'Retail Trade', 'Transit/Warehouse', 
    'Info', 'Finance/Insurance', 'Real Estate', 'Rental', 'Prof/Tech', 'Mgmt', 'Admin', 'Educ', 
    'Health', 'Arts', 'Accom', 'Food Services', 'Other Services'
]

In [29]:
len(sectors)

41

In [30]:
invnet = io.loadmat('./investmentNetwork/Replication Packet/Investment Network Construction/invmatdat_41.mat')

In [31]:
avg_net = pd.DataFrame(np.mean(invnet['invmat'], axis=2))
avg_net.columns = sectors
avg_net.index = sectors

The investment network was constructed based on sector-level asset investments and production using the below equation:

$$ I_{ijt} = \sum_{a=1}^{A}\omega_{iat} I_{ajt}^{exp}$$

Where $I$ is the (37x37x72) investment matrix for 1947-2018. $I_{ajt}^{exp}$ is the expenditure by sector $j$ on asset $a$ in year $t$. $\omega_{iat}$ is the fraction of asset $a$ produced by sector $i$ in year $t$. The network below is the mean (37x37) matrix across time. 

The original network relies on a number of assumptions that largely contribute to the investment hubs it identified. These assumptioins are mostly due to the lack of data on sector-level asset production.

1. Construction sector produces all non-mining structures
2. Prof/Technical Services produce all custom software and R&D
3. Artistic originals are only produced by info/comm or arts
4. Info/comm produce all pre-packaged software but not delivery
5. Wood Manufacturing, insurance/finance, and Prof/Tech services produce residential structures but not non-residential structures

In addition to these theoretic assumptions, much of the data pre-1997 is estimated bluntly by taking the average production/expenditure ratios post-1997 or interpolating with data from 1987 and 1992.

Moreover, the asset expenditure data was flagged as unreliable by BEA and it was specifically said they're "more likely to be based on judgemental trends, on trends in the higher level aggregate, or on less reliable source data." On the other hand, the asset production data is mostly just for "equipment" and the allocations are based on the assumptions above.

Are there asset classes baked into the macro model? If so, that would be a natural set of assets to base an investment network off of

In [32]:
fig = px.imshow(avg_net)
fig.update_layout(
    autosize=False,
    width=800,
    height=800
)
fig.show()

### Simple Version of Investment Network

In this network, I aggregate the sectors into 18 that map more cleanly between BEA codes and the ISIC codes and attempt to construct an investment network using the same methods as VA. The loss is the granularity of the data into 37+ sectors and I will only use 1997+ data.

### Attempt at Mapping BEA -> NAICS -> ISIC/NACE

One of the issues with the mapping between NAICS and ISIC is that both the Investment Network paper and the macro model use intermediate aggregates of the coding scheme while the mapping is on the lowest-level. I haven't found any mappings that map this intermediate level so I have to map from NAICS-mid -> NAICS-low -> ISIC-low -> ISIC-mid. Which is a many-to-many mapping. 

The approach I'm taking above maps the highest-level aggregate sectors into 18 relatively clean-to-match sectors and reconstructs an investment matrix for these using VW's approach.

In [34]:
# compares to "Replication Packet/Converting SIC to NAICS/SIC_NAICS_BEA_allsec.do"

NAICS_groups = {
    'Agriculture': [111,112,113,114,115], # excluded
    'Mining': [212],
    'Oil/Gas': [211], # excluded
    'Mining Support': ['212'], # excluded
    'Util': [22], # 221 doesn't exist in NAICS codes in asset expenditures but in VW mapping
    'Const': [23],
    'Wood': [321],
    'Minerals': [327],
    'Primary Metals': [331],
    'Fabricated Metals': [332],
    'Machinery': [333],
    'Computers': [334],
    'Electrical': [335],
    'Vehicles': [3361,3362,3363],
    'Transport': [3364,3365,3366,3367,3368,3369],
    'Furniture': [337],
    'Misc Mfg': [339],
    'Food Mfg': [311,312],
    'Textile': [313,314],
    'Apparel': [315,316],
    'Paper': [322],
    'Printing': [323],
    'Petroleum': [324],
    'Chemical': [325],
    'Plastics': [326],
    'Wholesale Trade': [42],
    'Retail Trade': [44,45],
    'Transit/Warehouse': [48,49],
    'Info': [51],
    'Finance/Insurance': [52],
    'Real Estate': [531],
    'Rental': [532,533], # excluded
    'Prof/Tech': [54],
    'Mgmt': [55],
    'Admin': [561,562], # just 56 in the VW mapping
    'Educ': [61],
    'Health': [62],
    'Arts': [71],
    'Accom': [721],
    'Food Services': [722],
    'Other Services': [81]
}

In [35]:
naics_isic_df = pd.read_csv('NAICS_ISIC.csv', dtype={'NAICS2012Code': str, 'ISIC4Code': str})

naics_isic_map = {}
for i, row in naics_isic_df.iterrows():
    naics_isic_map[str(row[0])] = row[2]
    

In [36]:
ISIC_groups = {}
for key, ids in NAICS_groups.items():
    ISIC_groups[key] = []
    for i in ids:
        for naics_code, isic_code in naics_isic_map.items():
            id_len = len(str(i))
            if not naics_code is None and naics_code[0:id_len] == str(i):
                ISIC_groups[key].append(str(isic_code))
    ISIC_groups[key] = list(set(ISIC_groups[key]))

In [37]:
isic_nace_df = pd.read_csv('ISIC_NACE.csv', dtype={'ISIC4code': str, 'NACE2code': str})

isic_nace_map = {}
for i, row in isic_nace_df.iterrows():
    isic_nace_map[str(row[0])] = row[2]

In [38]:
NACE2_groups = {}
for key, ids in ISIC_groups.items():
    NACE2_groups[key] = []
    for i in ids:
        for isic_code, nace_code in isic_nace_map.items():
            id_len = len(str(i))
            if not isic_code is None and isic_code[0:id_len] == str(i):
                NACE2_groups[key].append(str(nace_code))
    NACE2_groups[key] = np.sort(list(set(NACE2_groups[key])))

In [39]:
# mining == mining support (08), fabricated metals == furniture (25)
NACE2_groups_agg = {key: pd.Series([x[0:2] for x in val]).value_counts().index for key, val in NACE2_groups.items()} 
# mining == mining support (08), minerals == petroleum (23)
ISIC_groups_agg = {key: pd.Series([x[0:2] for x in val]).value_counts().index for key, val in ISIC_groups.items()}

In [40]:
sea_df = pd.read_csv('sample_data/Socio_Economic_Accounts.csv')

our_map = {}
for i, row in sea_df.iterrows():
    our_map[row[2]] = row[3]

In [41]:
def clean_isic(x):
    ids = re.findall('\d{2}', x)
    if len(ids) == 0:
        return [x]
    elif len(ids) == 1:
        return [int(x[1:])]
    else:
        int_ids = [int(id) for id in ids]
        min = np.min(int_ids)
        max = np.max(int_ids)
        return list(np.arange(min, max))
    
clean_our_map = {}
for key in our_map.keys():
    clean_our_map[key] = clean_isic(our_map[key])

In [42]:
isic_ours_map = {}
for isic in ISIC_groups_agg.items():
    isic_ours_map[isic[0]] = []
    for ours in clean_our_map.items():
        if len(set(ours[1]).intersection(set([int(i) for i in isic[1]]))) > 0:
            isic_ours_map[isic[0]].append(our_map[ours[0]])

In [43]:
isic_ours_map_manual_edit = {
    'Agriculture': ['A01', 'A02', 'A03', 'C10-C12', 'C16'],
    'Mining': ['B'],
    'Oil/Gas': [],
    'Mining Support': ['B'],
    'Util': ['D35', 'E37-E39', 'H49'],
    'Const': ['F'],
    'Wood': ['C16', 'C31_C32'],
    'Minerals': ['C23'],
    'Primary Metals': ['C24', 'C25', 'C27', 'C28', 'C30'],
    'Fabricated Metals': ['C24', 'C25', 'C28', 'C29', 'C30'],
    'Machinery': ['C25', 'C26', 'C28', 'C30'],
    'Computers': ['C18', 'C26', 'C30'],
    'Electrical': ['C27'],
    'Vehicles': ['C25', 'C29', 'C30'],
    'Transport': ['C30', 'C33'],
    'Furniture': ['C25', 'C31_C32'],
    'Misc Mfg': ['C27', 'C28'],
    'Food Mfg': ['C10-C12', 'C20', 'D35'],
    'Textile': ['C13-C15', 'C22'],
    'Apparel': ['C13-C15', 'C22'],
    'Paper': ['C17', 'C23', 'C25'],
    'Printing': ['C18'],
    'Petroleum': ['C19', 'C23'],
    'Chemical': ['C20', 'C21', 'C22', 'C28'],
    'Plastics': ['C22', 'C27', 'C31_C32'],
    'Wholesale Trade': ['G45', 'G46'],
    'Retail Trade': ['G45', 'G47'],
    'Transit/Warehouse': ['H49', 'H50', 'H51', 'H52', 'H53'],
    'Info': ['J58', 'J59_J60', 'J61', 'M74_M75'],
    'Finance/Insurance': ['K64', 'K65', 'K66', 'L68'],
    'Real Estate': ['L68'],
    'Rental': [],
    'Prof/Tech': ['J62_J63', 'M69_M70', 'M71', 'M72', 'M73', 'M74_M75'],
    'Mgmt': ['K64'],
    'Admin': ['E37-E39', 'O84'],
    'Educ': ['P85'],
    'Health': ['Q'],
    'Arts': ['M74_M75'],
    'Accom': ['I'],
    'Food Services': ['I'],
    'Other Services': ['C33', 'G45', 'M71', 'M74_M75']
 }

In [44]:
avg_net = pd.DataFrame(np.mean(invnet['invmat'], axis=2))
selection = [rnd.choice(isic_ours_map_manual_edit[s]) for s in sectors]
avg_net.columns = selection
avg_net.index = selection

IndexError: Cannot choose from an empty sequence

In [ ]:
fig = px.imshow(avg_net)
fig.update_layout(
    autosize=False,
    width=800,
    height=800
)
fig.show()

In [ ]:
avg_net = pd.DataFrame(np.mean(invnet['invmat'], axis=2))
avg_net.columns = sectors
avg_net.index = sectors

In [ ]:
fig = px.imshow(avg_net)
fig.update_layout(
    autosize=False,
    width=800,
    height=800
)
fig.show()

## Quick and Dirty Mapping

This maps and aggregates the VW investment matrix into the ISIC high-level sectors (A-U).

In [ ]:
agg_sectors = {
 'A': ['Agriculture'],
 'B': ['Mining', 'Oil/Gas', 'Mining Support'],
 'C': ['Wood',
  'Minerals',
  'Primary Metals',
  'Fabricated Metals',
  'Machinery',
  'Computers',
  'Electrical',
  'Vehicles',
  'Transport',
  'Furniture',
  'Misc Mfg',
  'Food Mfg',
  'Textile',
  'Apparel',
  'Paper',
  'Printing',
  'Petroleum',
  'Chemical',
  'Plastics',
  'Other Services'],
 'D': ['Util'],
 'E': ['Admin'],
 'F': ['Const'],
 'G': ['Wholesale Trade', 'Retail Trade'],
 'H': ['Transit/Warehouse'],
 'I': ['Accom', 'Food Services'],
 'J': ['Info'],
 'K': ['Finance/Insurance', 'Mgmt'],
 'L': ['Real Estate'],
 'M': ['Prof/Tech'],
 'N': ['Rental', 'Admin'],
 'P': ['Educ'],
 'Q': ['Health'],
 'R': ['Arts'],
 'S': ['Other Services']}

# avg_net = pd.DataFrame(np.mean(invnet['invmat'], axis=2))


In [ ]:
avg_net = pd.DataFrame(np.mean(invnet['invmat'][:,:,-21:], axis=2))
avg_net.columns = sectors
avg_net.index = sectors

In [ ]:
rows = []

# sum row-wise
for key, vals in agg_sectors.items():
    if len(vals) > 0:
        new_row = {s: 0 for s in sectors}
        new_row['sector'] = key
        for i, row in avg_net.iterrows():
            if i in vals:
                for i in range(len(sectors)):
                    new_row[sectors[i]] = new_row[sectors[i]] + row[i]
        rows.append(new_row)

In [ ]:
row_summed_df = pd.DataFrame(rows)
summed_df = row_summed_df.copy()

sector_letters = []

# this doesn't take relative sub sector sizes into account *very rought*
# mean column-wise
for key, vals in agg_sectors.items():
    if len(vals)> 0:
        sector_letters.append(key)
        summed_df[key] = np.mean(row_summed_df[vals], axis=1)

rough_matrix = summed_df[sector_letters]
rough_matrix.index = sector_letters
rough_matrix.to_csv('rough_invmat.csv')

In [ ]:
fig = px.imshow(rough_matrix)
fig.update_layout(
    autosize=False,
    width=800,
    height=800
)
fig.show()

### Limitations

It's essentially just grouping the sub-sectors, summing row-wise and taking the mean column-wise. Which means it's not weighted for sub-sector size (e.g. if wholesale trade and retail trade purchase 10% and 20% of their total expenditures from manufacturing, G which is their combined sector, will have 15% expenditure to manufacturing regardless of relative size).

In addition, it's missing A (agriculture), E (waste management), O (public admin), T (household activities), U (extraterritorial) since the paper was focused on non-farm firms. If I were to repeat their process using the full data, I could add these to the matrix but that would take a bit more time.

## Network Recreation

For now, using the same data that VW use to create the network.

In [117]:
expenditure_data = './assetInvestments.xlsx'

In [118]:
readme = pd.read_excel(expenditure_data, header=14, converters={'BEA CODE': str, '2012 NAICS Codes': str})
readme = readme[['BEA CODE', '2012 NAICS Codes']][readme['BEA CODE'].map(lambda x: not (type(x) is float or x == '--------') )]
bea_naics_map = {}
for i, row in readme.iterrows():
    naics = row[1].split(',')
    for n in naics:
        if '-' in naics[0]:
            temp = naics[0].split('-')
            print(temp)
            naics = [str(x) for x in np.arange(int(temp[0]),int(temp[0][:-1] + temp[1]) + 1)]
    bea_naics_map[row[0]] = [n.strip() for n in naics]

['3361', '3']
['3364', '9']
['44', '5']


In [119]:
pd.set_option('display.max_rows', 200)

class InvestmentNetworkPipe:

    def __init__(self, path, industries):

        dfs = []
        for ind in industries:
            dfs.append(np.array(self.extract_expenditure_table(path, ind).iloc[:,3:]))

        dfs = np.array(dfs)

        yearly_dfs = []
        for year in range(dfs.shape[2]):
            yearly_dfs.append(dfs[:,:,year])

        yearly_dfs = np.array(yearly_dfs)

        full_tab = self.extract_expenditure_table(path, ind)

        self.matrix = yearly_dfs
        self.industries = industries
        self.assets = list(full_tab['NIPA Asset Types'])
        self.asset_codes = list(full_tab['Asset Codes'])

    def extract_expenditure_table(self, path,industry):

        df = pd.read_excel(path, header=5, sheet_name=industry).iloc[2:].reset_index(drop=True)
        types = ['equipment' for _ in range(39)] + ['structures' for _ in range(32)] + ['intellectual' for _ in range(25)]
        df.drop([39,72], axis=0, inplace=True)
        df.drop([str(y) for y in range(1901,2000)], axis=1, inplace=True)
        df['type'] = types
        df = df[['type'] + list(df.columns[0:-1])]

        return df.reset_index(drop=True)
    
    def get_year(self, year, code=True, sectoral=True):
        
        df = pd.DataFrame(self.matrix[year % 2000])
        print(df.shape)
        
        if code: df.columns = self.asset_codes
        else: df.columns = self.assets
        df.index = self.industries

        return df

pipe = InvestmentNetworkPipe(expenditure_data, list(bea_naics_map.keys())[:-1])

In [125]:

def convert_to_isic(pipe, year):
        
    with open('bea_isic_map.json', 'r') as f:
        bea_isic = json.loads(f.read())

    # gets rid of "other services, except government"
    one_year = pipe.get_year(year, code=True).iloc[:,:-1]
    one_year.index = [bea_isic[i] for i in list(one_year.index)]

    rows = []
    for sector in np.sort(np.unique(one_year.index)):

        if len(one_year.loc[sector].shape) > 1:
            row = { 'sector': sector, **dict(one_year.loc[sector].sum(axis=0))}
        else:
            row = { 'sector': sector, **dict(one_year.loc[sector])}
            
        rows.append(row)

    return pd.DataFrame(rows)

convert_to_isic(pipe, 2006)

(62, 96)


,sector,EP1A,EP1B,EP1C,EP1D,EP1E,EP1F,EP1G,EP1H,EP20,...,RD40,RD50,RD60,RD80,RD91,RD92,AE10,AE20,AE30,AE40
0,A,5.0,67.0,0.0,24.0,15.0,0.0,12.0,39.0,109.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,B,48.0,420.0,0.0,70.0,65.0,0.0,50.0,150.0,962.0,...,0.0,0.0,0.0,1243.0,0.0,0.0,0.0,0.0,0.0,0.0
2,C,427.0,3823.0,0.0,806.0,767.0,0.0,528.0,1615.0,4323.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,D,61.0,461.0,0.0,51.0,49.0,0.0,76.0,84.0,1058.0,...,0.0,0.0,0.0,237.0,0.0,0.0,0.0,0.0,0.0,0.0
4,E,9.0,79.0,0.0,17.0,14.0,0.0,12.0,27.0,184.0,...,0.0,0.0,0.0,69.0,0.0,0.0,0.0,0.0,0.0,0.0
5,F,65.0,619.0,0.0,147.0,137.0,0.0,150.0,552.0,883.0,...,0.0,0.0,0.0,1515.0,0.0,0.0,0.0,0.0,0.0,0.0
6,G,2346.0,4224.0,0.0,1723.0,1723.0,0.0,1565.0,2347.0,4799.0,...,0.0,0.0,0.0,3793.0,0.0,0.0,0.0,0.0,0.0,0.0
7,H,908.0,849.0,0.0,191.0,190.0,0.0,347.0,315.0,3625.0,...,0.0,0.0,0.0,306.0,0.0,0.0,0.0,0.0,0.0,0.0
8,I,66.0,560.0,0.0,215.0,206.0,0.0,147.0,422.0,462.0,...,0.0,0.0,0.0,398.0,0.0,0.0,0.0,0.0,0.0,0.0
9,J,636.0,5115.0,0.0,975.0,934.0,0.0,1039.0,2062.0,56439.0,...,19059.0,0.0,0.0,7872.0,0.0,0.0,20060.0,30823.0,6344.0,4235.0
